In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm, tqdm_notebook
import gc
import json

In [2]:
df = pd.read_json('embedding.json', orient='index', convert_axes=False)
df.index = df.index.astype(int)
df

,isbn,title,author,description,embedding
0,9771197225987,흘러가는 구름을 동경하였다,"김시탁, 김일태, 민창홍, 성선경, 이강휘, 이기영, 이달균, 이서린, 이월춘",시문학연구회 하로동선의 여섯 번째 동인지 지금까지 현대시의 새로운 방향성을 탐구해 ...,"[0.020481575300000002, -0.024692751500000002, ..."
1,9772586200006,장애와 테크놀로지 - 과학잡지 에피 16호,"김초엽, 김원영, 김상희, 유기훈, 이주현, 류성두, 명수민, 킴 포춘, 전치형, ...",에피 16호 키워드 숨 섹션은 장애와 테크놀로지를 주제로 삼았다 기술의 발전이 장애...,"[0.0539464802, 0.0389155224, -0.0651673153, -0..."
2,9772586200211,식물의 과학 - 과학잡지 에피 17호,"강연실, 허준이, 금종해, 안수지, 이상엽, 남재환, 황승식, 장진성, 박찬열, 강...",과학비평 계간지 에피 17호는 식물의 과학을 다루며 17편의 글을 담았다 식물을 나...,"[0.0425348207, -0.001016357, 0.0275022872, -0...."
3,9772733807003,"에픽 #04 - BELOVED, 2021.7.8.9","임현, 임철홍, 에밀리 정민 윤, 이정식, 임소라, 김대성, 박하빈, 염은영, 김혜...",4호의 제호는 이다 1988년 퓰리처상 수상작이기도 한 토니 모리슨의 장편 빌러비드...,"[-0.0164510254, -0.0326858386, -0.030496591700..."
4,9772765656006,오늘의 좋은 소설 2021.가을 - 58호,"김현, 서정아, 이미욱, 이태형, 정미형",오늘의 좋은 소설은 간단하다 무겁지도 어렵지도 않다 어떤 장식과 해설도 하지 않고 ...,"[-0.0835911185, -0.0344673432, -0.0317333005, ..."
...,...,...,...,...,...
5954,9791197556609,왓더? 오디오 1,"최윤욱, 김연승, 최창림 (디자인)",만화는 여주인공인 수진이가 이어폰으로 음악을 듣다가 귀에 이명이 와서 스피커를 통해...,"[0.014301928700000001, 0.0657247379, -0.073053..."
5955,9791197558009,오늘의 어린이책 1 - 다움북클럽이 고른 성평등 어린이·청소년책 2019-2021,다움북클럽,다시 시작하는 다움북클럽이 고른 성평등 어린이청소년책 목록 20192021은 기존 ...,"[0.0398772843, 0.0599167943, -0.012749128, -0...."
5956,9791197559105,"괜찮아, 아빠도 쉽진 않더라 - 오늘을 살아가는 우리 모두를 위한 동화, &lt;퇴...",문수림,여기 마흔 한 살에 아빠가 된 무명 작가가 있다 대중에게 인식되지 못할 정도의 글솜...,"[0.0164277479, 0.0226795059, -0.0629340634, -0..."
5957,9791197566400,미래를 향한 노스탤지어,신미로,불길한 죽음의 예감 때문에 시작된 여행 그 수많은 여행에서 내가 가장 멀리까지 가본...,"[0.0246997848, 0.0388689004, 0.0501789376, -0...."


In [3]:
array_embedding = np.array([np.array(xi) for xi in df['embedding']])
array_embedding

print(array_embedding)
print(array_embedding.shape)

[[ 0.02048158 -0.02469275  0.01219742 ... -0.00748551  0.04424236
  -0.03050544]
 [ 0.05394648  0.03891552 -0.06516732 ...  0.02442933  0.00083294
   0.0647038 ]
 [ 0.04253482 -0.00101636  0.02750229 ...  0.02031228  0.0417028
   0.01048672]
 ...
 [ 0.01642775  0.02267951 -0.06293406 ... -0.0275322   0.01551971
  -0.01796408]
 [ 0.02469978  0.0388689   0.05017894 ... -0.05910479 -0.07302566
   0.05451297]
 [-0.01611387 -0.01039436 -0.03350398 ...  0.01053391 -0.05075981
  -0.03108358]]
(5959, 512)


In [4]:
cosine_scores = cosine_similarity(array_embedding, array_embedding)
print(cosine_scores)
print(cosine_scores.shape)

[[1.         0.14435119 0.17866998 ... 0.09797161 0.15817259 0.19873803]
 [0.14435119 1.         0.26626255 ... 0.12936234 0.0892551  0.17847842]
 [0.17866998 0.26626255 1.         ... 0.15882838 0.14266754 0.18743436]
 ...
 [0.09797161 0.12936234 0.15882838 ... 1.         0.26299183 0.34600902]
 [0.15817259 0.0892551  0.14266754 ... 0.26299183 1.         0.12055081]
 [0.19873803 0.17847842 0.18743436 ... 0.34600902 0.12055081 1.        ]]
(5959, 5959)


In [5]:
# test printing
idx = np.argsort(-cosine_scores[0])[:6][1:]
print(idx)
print(cosine_scores[0][idx])
print(df.loc[0, ['title', 'description']])
print(df.loc[df.index.isin(idx), ['title', 'description']])

[2253 5201 4977 5126 4767]
[0.5074089  0.47227948 0.4553376  0.44046396 0.43540814]
title                                             흘러가는 구름을 동경하였다
description    시문학연구회 하로동선의 여섯 번째 동인지 지금까지 현대시의 새로운 방향성을 탐구해 ...
Name: 0, dtype: object
                                              title  \
2253                                   빨간 치마를 입은 아이   
4767             모든 이의 과학사 강의 - 역사와 문화로 이해하는 과학 인문학   
4977                        소비자행동 - 마케팅 전략적 접근, 제7판   
5126                                  삽화와 함께 읽는 도련님   
5201  회사는 싫지만 회사 생활은 잘하고 싶어 - 요즘 직장인을 위한 12가지 생존 기술   

                                            description  
2253  2018년 문화일보 신춘문예로 등단한 이경란의 첫 소설집  작가의 항해와 탐색은 관...  
4767  앎이 삶을 바꾼다는 관점에서 인간의 탄생에서 현대의 눈부신 과학기술의 발전과 그 명...  
4977  다른 모든 학문 분야와 마찬가지로 소비자행동에 관한 연구는 지속적으로 발전되고 있다...  
5126  일본의 대문호이자 현대문학의 개척자인 나쓰메 소세키의 대표작 도련님이 만화의 개척자...  
5201  신입부터 시작해 6년 만에 가 된 직장의 신 추이추이의 회사 생활 노하우를 모두 담...  


In [34]:
def sort_cosine_score_top_n(n):
    for i, row in enumerate(cosine_scores):
        idx = list(np.argsort(-row)[:n+1] )# i번째 행의 상위 n개 인덱스
        idx.remove(i)
        print('----'*9, 'title:', df.loc[i, ['title', 'description']].values)
        print('----'*9, 'top idx:', idx)
        print('\n')
        print(df.loc[df.index.isin(idx), ['title', 'description']])
        print('\n')
        print('----'*30)
        if i == 10:
            break
        

sort_cosine_score_top_n(5)


------------------------------------ title: ['흘러가는 구름을 동경하였다'
 '시문학연구회 하로동선의 여섯 번째 동인지 지금까지 현대시의 새로운 방향성을 탐구해 온 바 있으며 이번 제6집 동인지에서도 이러한 활동은 계속된다 지금껏 이어져온 현대시 정신을 수용하면서도 새로운 길을 탐구하는 노력이 면면하다']
------------------------------------ top idx: [2253, 5201, 4977, 5126, 4767]


                                              title  \
2253                                   빨간 치마를 입은 아이   
4767             모든 이의 과학사 강의 - 역사와 문화로 이해하는 과학 인문학   
4977                        소비자행동 - 마케팅 전략적 접근, 제7판   
5126                                  삽화와 함께 읽는 도련님   
5201  회사는 싫지만 회사 생활은 잘하고 싶어 - 요즘 직장인을 위한 12가지 생존 기술   

                                            description  
2253  2018년 문화일보 신춘문예로 등단한 이경란의 첫 소설집  작가의 항해와 탐색은 관...  
4767  앎이 삶을 바꾼다는 관점에서 인간의 탄생에서 현대의 눈부신 과학기술의 발전과 그 명...  
4977  다른 모든 학문 분야와 마찬가지로 소비자행동에 관한 연구는 지속적으로 발전되고 있다...  
5126  일본의 대문호이자 현대문학의 개척자인 나쓰메 소세키의 대표작 도련님이 만화의 개척자...  
5201  신입부터 시작해 6년 만에 가 된 직장의 신 추이추이의 회사 생활 노하우를 모두 담...  


------------------------------------------------

In [29]:
# pairs = []
# for i in tqdm(range(len(cosine_scores)-1)):
#     for j in range(i+1, len(cosine_scores)):
#         pairs.append({'isbn1': df.at[f'{i}', 'isbn'], 'isbn2': df.at[f'{j}', 'isbn'], 'score': cosine_scores[i][j]})

100%|██████████| 5958/5958 [02:01<00:00, 48.89it/s] 


In [46]:
df.loc[df['isbn'] == i, 'title'].values[0]

'파운데이션'

In [59]:
for i, n in tqdm(enumerate())

pairs = sorted(pairs, key=lambda x: (x['isbn1'], x['score']), reverse=True)


for pair in pairs[0:30]:
    i, j = pair['isbn1'], pair['isbn2']
    print(f"{df.loc[df['isbn'] == i, 'isbn'].values[0]}: {df.loc[df['isbn'] == i, 'title'].values[0]} \t\t")
    print(f"{df.loc[df['isbn'] == j, 'isbn'].values[0]}: {df.loc[df['isbn'] == j, 'title'].values[0]} \t\t")
    print(f" ㄴ Score: {pair['score']:.4f} ----------")

9791197566400: 미래를 향한 노스탤지어 		
9791197578410: 채사장의 지대넓얕 1 : 권력의 탄생 - 지적 대화를 위한 넓고 얕은 지식 		
 ㄴ Score: 0.1206 ----------
9791197559105: 괜찮아, 아빠도 쉽진 않더라 - 오늘을 살아가는 우리 모두를 위한 동화, &lt;퇴근 길 글 한잔!&gt; 		
9791197578410: 채사장의 지대넓얕 1 : 권력의 탄생 - 지적 대화를 위한 넓고 얕은 지식 		
 ㄴ Score: 0.3460 ----------
9791197559105: 괜찮아, 아빠도 쉽진 않더라 - 오늘을 살아가는 우리 모두를 위한 동화, &lt;퇴근 길 글 한잔!&gt; 		
9791197566400: 미래를 향한 노스탤지어 		
 ㄴ Score: 0.2630 ----------
9791197558009: 오늘의 어린이책 1 - 다움북클럽이 고른 성평등 어린이·청소년책 2019-2021 		
9791197578410: 채사장의 지대넓얕 1 : 권력의 탄생 - 지적 대화를 위한 넓고 얕은 지식 		
 ㄴ Score: 0.3209 ----------
9791197558009: 오늘의 어린이책 1 - 다움북클럽이 고른 성평등 어린이·청소년책 2019-2021 		
9791197559105: 괜찮아, 아빠도 쉽진 않더라 - 오늘을 살아가는 우리 모두를 위한 동화, &lt;퇴근 길 글 한잔!&gt; 		
 ㄴ Score: 0.2482 ----------
9791197558009: 오늘의 어린이책 1 - 다움북클럽이 고른 성평등 어린이·청소년책 2019-2021 		
9791197566400: 미래를 향한 노스탤지어 		
 ㄴ Score: 0.0922 ----------
9791197556609: 왓더? 오디오 1 		
9791197578410: 채사장의 지대넓얕 1 : 권력의 탄생 - 지적 대화를 위한 넓고 얕은 지식 		
 ㄴ Score: 0.3725 ----------
9791197556609: 왓더

In [10]:
# filename = 'cosine_scores_transformer.json'

# with open(filename, 'w', encoding='utf-8') as f:
#     json.dump([], f)

# for i in tqdm(range(len(cosine_scores)-1)):
#     with open(filename, 'r', encoding='utf-8') as f:
#         pairs = json.load(f)
#     for j in range(i+1, len(cosine_scores)):
#         pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
#     with open(filename, 'w', encoding='utf-8') as f:
#        json.dump(pairs, f, ensure_ascii=False)
       


  0%|          | 10/5958 [00:02<32:22,  3.06it/s]Exception ignored in: <function tqdm.__del__ at 0x7faa8b948af0>
Traceback (most recent call last):
  File "/home/kok/anaconda3/envs/rjs/lib/python3.8/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/home/kok/anaconda3/envs/rjs/lib/python3.8/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
  0%|          | 27/5958 [00:22<1:23:57,  1.18it/s]


KeyboardInterrupt: 

In [4]:
with open('cosine_scores_transformer.json', 'r', encoding='utf-8') as f:
    cosine_scores_transformer = json.load(f)
cosine_scores_transformer